In [1]:
import openai
from openai import OpenAI

In [2]:
client = OpenAI()

In [3]:
response = client.chat.completions.create(
    model = 'gpt-4o',
    messages = [{"role" : "user", "content": "is it too late to join the course?"}]
)

In [4]:
response

ChatCompletion(id='chatcmpl-9ZwnxgGd64G5E30oIOU7gH7ZTi2y5', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="Whether it's too late to join a course depends on several factors:\n\n1. **Enrollment Deadlines:** Each course or institution usually has specific enrollment deadlines.\n2. **Course Start Date:** If the course has already started, some institutions may still allow late registration within a certain period.\n3. **Availability:** There might be limited spots in the course, and it might be full.\n4. **Instructor Policies:** Some instructors or programs may accept late enrollees, especially if the student can catch up on missed material.\n\nYou should check with the course provider or institution directly to get specific information about whether you can still join. They will provide details about deadlines, availability, and any potential consequences or requirements for late enrollment.", role='assistant', function_call=None, too

In [5]:
response.choices[0].message.content

"Whether it's too late to join a course depends on several factors:\n\n1. **Enrollment Deadlines:** Each course or institution usually has specific enrollment deadlines.\n2. **Course Start Date:** If the course has already started, some institutions may still allow late registration within a certain period.\n3. **Availability:** There might be limited spots in the course, and it might be full.\n4. **Instructor Policies:** Some instructors or programs may accept late enrollees, especially if the student can catch up on missed material.\n\nYou should check with the course provider or institution directly to get specific information about whether you can still join. They will provide details about deadlines, availability, and any potential consequences or requirements for late enrollment."